In [ ]:
import glob
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from src.datagenerator import DataGenerator, DataLoader, z_score
from src.performance_evaluation import (
    compute_PCA,
    compute_position_error,
    compute_volume_error,
    plot_confusion_matrix,
    plot_PCA,
)
from src.util import plot_voxel

# load vae model
from src.vae import vae_model

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
print(tf.__version__)

# Stage 2

**1. Training of the material classificator $\Upsilon$**

**2. Training of the mapper $\Xi$**

## Material classificator $\Upsilon$

**CNN**

In [ ]:
datagenerator = False

params = {
    "path": "../3dIT/measurements/datapool/",
    "mean_path": "../3dIT/measurements/datameans/",
    "eit_dim": 4096,
    "supervised": "material",  # "diameter", "material", "anomaly", "anomaly_and_material"
    "batch_size": 16,
    "shuffle": True,
    "EIT_shape": "matrix",
}

if datagenerator:
    idx = np.arange(0, len(os.listdir(params["path"])))
    np.random.shuffle(idx)

    limit_index = 100_000

    training_generator = DataGenerator(idx[:limit_index], **params)
    test_____generator = DataGenerator(idx[limit_index:], **params)
    X, Y = training_generator[0]
else:
    X, Y = DataLoader(params)
    X = z_score(X)
    X = np.expand_dims(X, axis=3)
    print(X.shape, Y.shape)

In [ ]:
print(X.shape, Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=42
)

In [ ]:
data_pca = compute_PCA(X_train)
plot_PCA(data_pca, Y_train)

In [ ]:
data_pca = compute_PCA(X_test)
plot_PCA(data_pca, Y_test)

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
def material_classificator(input_shape=(64, 64, 1), m_dim=1, kernel=3):
    mapper_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(8, kernel, strides=(2, 4), padding="same")(mapper_input)
    x = tf.keras.layers.Conv2D(16, kernel, strides=(2, 4), padding="same")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(16)(x)
    mapper_output = tf.keras.layers.Dense(m_dim, activation="sigmoid")(x)
    return tf.keras.Model(mapper_input, mapper_output)


material_classificator_CNN = material_classificator()
material_classificator_CNN.summary()

It is possible to insert the `softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy
material_classificator_CNN.compile(
    tf.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
classificator_history_CNN = material_classificator_CNN.fit(
    X_train,
    Y_train,
    epochs=20,
    batch_size=params["batch_size"],
)

In [ ]:
plt.plot(classificator_history_CNN.history["loss"])

In [ ]:
material_classificator_CNN.save_weights("models/material_mapper.weights.h5")

In [ ]:
Y_pred = np.concatenate(np.round(material_classificator_CNN.predict(X_test)))

In [ ]:
plot_confusion_matrix(Y_test, Y_pred)

In [ ]:
# os.mkdir(s_path)
# material_classificator_CNN.save(f"{s_path}/model.keras")
# np.savez(s_path + "/test_data.npz", X_test=X_test, Y_test=Y_test, params=params)

## Mapper $\Xi$

Loat the VAE model and train the two required mapper.

In [ ]:
vae = vae_model()
vae.load_weights("models/vaes/vae_21.weights.h5")
vae.summary()

Φ = vae.encoder
Ψ = vae.decoder

In [ ]:
params = {
    "path": "../3dIT/measurements/datapool/",
    "mean_path": "../3dIT/measurements/datameans/",
    "eit_dim": 4096,
    "EIT_shape": "matrix",
    "supervised": "anomaly_and_material",  # "diameter", "material", "anomaly", "anomaly_and_material"
    "batch_size": 128,
    "shuffle": False,
}

In [ ]:
X, Y, m = DataLoader(params)

X = z_score(X)
X = np.expand_dims(X, axis=3)
Y = np.expand_dims(Y, axis=4)

In [ ]:
X_train, X_test, gamma_train, gamma_test, m_train, m_test = train_test_split(
    X, Y, m, test_size=0.1, random_state=42
)
# del X,Y,m

In [ ]:
_, _, z_train = Φ.predict(gamma_train)

In [ ]:
_, _, z_test = Φ.predict(gamma_test)

In [ ]:
print(X_train.shape, gamma_train.shape, m_train.shape, z_train.shape)
print(X_test.shape, gamma_test.shape, m_test.shape, z_test.shape)

In [ ]:
tmp = np.load("models/mappers/predictable_model_10.npz", allow_pickle=True)
print(f"{tmp['precision']:.2f}")

In [ ]:
# |---------------------- HPT#1 --------------------- with VAE model = 17-------------------------------------------------------------|
# | model  a1: e = 50 , bs = 16, filters=[2,  4,  8, 16], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=55.06 %|
# | model  a2: e = 75 , bs = 16, filters=[4,  8, 16, 32], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=50.93 %|
# | model  a3: e = 75 , bs = 16, filters=[4,  8,  8, 16], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=56.12 %|
# | model  a4: e = 75 , bs = 16, filters=[8,  8,  8, 16], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=61.84 %|
# | model  a5: e = 75 , bs = 16, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=58.16 %|

# | model  a6: e = 75 , bs =  8, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=77.92 %|
# | model  a7: e = 75 , bs =  8, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=67.13 %|
# | model  a8: e = 75 , bs =  8, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=64.90 %|
# | model  a9: e = 75 , bs =  8, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mae, predictable=62.13 %|
# | model a10: e = 75 , bs =  4, filters=[8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mae, predictable=55.17 %|
# |-----------------------------------------------------------------------------------------------------------------------------------|

# Changed the VAE because of the discovery of a better generalized version.
# ... continue with the best prediction performance of HPT#1 -> model configuration 6.

# |--------------------- HPT#2 -----------------------with VAE model = 21-------------------------------------------------------------|
# | model  1: e = 75 , bs =  8, filters=[ 8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=92.19 %|
# | model  2: e = 75 , bs =  6, filters=[ 8, 16, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=90.94 %|
# | model  3: e = 75 , bs =  8, filters=[16, 32, 32, 64], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=92.09 %|
# | model  4: e = 75 , bs =  8, filters=[8, 32, 64, 128], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable=92.77 %|
# | model  5: e = 75 , bs =  8, filters=[8, 32, 64, 128], kernels(2, 2), strides=(1, 1), pools=(2, 2), loss = mse, predictable=54.65 %|

# | model  6: e = 75 , bs =  8, filters=[4, 8, 16, 32], kernels(2, 2), strides=(1, 1), pools=(2, 2), loss = mse, predictable = 92.54 %|
# | model  7: e = 75 , bs =  6, filters=[4, 8, 16, 32], kernels(4, 4), strides=(1, 1), pools=(2, 2), loss = mse, predictable = 92.58 %|
# | model  8: e = 75 , bs =  5, filters=[4, 8, 16, 32], kernels(4, 4), strides=(1, 1), pools=(2, 2), loss = mse, predictable = 96.01 %|
# | model  9: e = 75 , bs =  16,filters=[4, 8, 16, 32], kernels(4, 4), strides=(1, 1), pools=(2, 2), loss = mse, predictable = 90.83 %|
# | model 10: e = 75 , bs =  5, filters=[4, 8, 16, 32], kernels(3, 3), strides=(1, 1), pools=(2, 2), loss = mse, predictable = 90.06 %|
# |-----------------------------------------------------------------------------------------------------------------------------------|

epochs = 75
batch_size = 5


def mapper_CNN(input_shape=(64, 64, 1), latent_dim=8):
    filters = [4, 8, 16, 32]
    kernels = [(3, 3) for _ in range(4)]
    strides = [(1, 1) for _ in range(4)]
    pools = [(2, 2) for _ in range(4)]

    print(f"{filters=}, kernels{kernels[0]}, strides={strides[0]}, pools={pools[0]}")

    x = tf.keras.layers.Input(shape=input_shape)
    mapper_input = x

    for f, k, s, p in zip(filters, kernels, strides, pools):
        x = tf.keras.layers.Conv2D(
            filters=f, kernel_size=k, strides=s, padding="valid", activation="relu"
        )(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=p)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(latent_dim, activation="relu")(x)
    mapper_output = tf.keras.layers.Dense(latent_dim)(x)

    return tf.keras.Model(mapper_input, mapper_output)


Ξ = mapper_CNN()
# losses to try:
# mean_squared_logarithmic_error
# mean_absolute_percentage_error
Ξ.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="mse")
Ξ.summary()

In [ ]:
print(f"{batch_size=}, {epochs=}")
history = Ξ.fit(X_train, z_train, batch_size=batch_size, epochs=epochs)

In [ ]:
plt.plot(history.history["loss"], label=keys)
plt.show()

**Test the mapper $\Xi$ performance**

The variable `γ_hat` represents $\hat{\gamma}$, which is the reconstructed conductivity distribution.
It is generated by:
$$\hat{\gamma} = \Psi(\Xi(\mathbf{u})) = \Psi \circ \Xi :  \mathbf{u} \mapsto \mathbf{z} \mapsto \hat{\gamma} $$

In [ ]:
γ_hat = Ψ.predict(Ξ.predict(X_test))
γ_hat = np.squeeze(γ_hat, axis=4)
γ_hat = np.clip(γ_hat, a_min=0, a_max=1)
γ_hat[γ_hat != 0] = 1
print(γ_hat.shape)

In [ ]:
plt.style.use("default")
for rdn in np.random.randint(low=0, high=γ_hat.shape[0], size=5):
    print("True γ distribution")
    plot_voxel(gamma_test[rdn, :, :, :, 0])
    print("Predicted γ distribution")
    plot_voxel(γ_hat[rdn])
    print("---------")

In [ ]:
mapper_idx = len(glob.glob("models/mappers/*.h5")) + 1
mapper_idx

In [ ]:
acc_measure = dict(Model=list(), Error_val=list(), Error=list(), test_idx=list())

data_append = []

for idx, v_err in enumerate(compute_volume_error(γ_hat, gamma_test)):
    if v_err != None:
        v_percent = (v_err / 32**3) * 100
        data_append.append(
            dict(Model=mapper_idx, Error_val=v_percent, Error="Volume", test_idx=idx)
        )

for idx, p_err in enumerate(compute_position_error(gamma_test, γ_hat)):
    if p_err != None:
        p_percent = (p_err / np.linalg.norm(np.array([32, 32, 32]))) * 100
        data_append.append(
            dict(Model=mapper_idx, Error_val=p_percent, Error="Position", test_idx=idx)
        )

# Create dictionary
for item in data_append:
    for key, value in item.items():
        acc_measure[key].append(value)

In [ ]:
acc_measure_df = pd.DataFrame(acc_measure).dropna()
acc_measure_df.to_csv(
    f"models/mappers/mappers_acc_measure_{mapper_idx}.csv", index=False
)
acc_measure_df

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
sns.set_context(context="paper", font_scale=1.4)
custom_palette = ["#756bb1", "#a1d99b"]
sns.boxplot(
    x="Model",
    y="Error_val",
    data=acc_measure_df,
    hue="Error",
    showfliers=True,
    palette=custom_palette,
)  #  violinplot: , split=True
plt.ylabel("Error (%)")
# plt.xlabel("Model Iteration")

In [ ]:
Ξ.save_weights(f"models/mappers/mapper_{mapper_idx}.weights.h5")

In [ ]:
# CHECK HOW MUCH TEST DATA CAN (NOT) BE PREDICTED!
volume_preds = len([True for ele in acc_measure_df["Error"] if ele == "Volume"])
position_preds = len([True for ele in acc_measure_df["Error"] if ele == "Position"])
print("Length condition is:", np.allclose(volume_preds, position_preds))
precision = volume_preds / gamma_test.shape[0] * 100
print(f"Copy this to the top hpt entry:\n {precision:.2f}%")
np.savez(
    f"models/mappers/predictable_model_{mapper_idx}.npz",
    precision=precision,
)

In [ ]:
# Load all CSV files and concatenate them
# plt.style.use('default')

combined_df = pd.concat(
    [
        pd.read_csv(f_csvs)
        for f_csvs in glob.glob("models/mappers/mappers_acc_measure_*.csv")
    ],
    ignore_index=True,
)
combined_df = pd.DataFrame(combined_df)
# combined_df.to_csv(f"models/mappers/all_measures.csv", index=False)
custom_palette = ["#756bb1", "#a1d99b"]
sns.boxplot(
    x="Model",
    y="Error_val",
    data=combined_df,
    hue="Error",
    showfliers=True,
    palette=custom_palette,
)  #  violinplot: , split=True
plt.ylabel("Error (%)")
plt.tight_layout()
# plt.savefig("images/vae_hpt.pdf")
plt.show()

___